In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from dateparser import parse
import datetime
from pathlib import Path
from dotenv import load_dotenv
from loguru import logger

#### Load environment variables containing username and password for Copernicus Scihub

In [ ]:
load_dotenv()
    
try:
    os.environ["DHUS_USER"]
except:
    logger.error("The environment variable 'DHUS_USER' is not set! Exiting...")
    raise KeyError("The environment variable 'DHUS_USER' is not set!")

try:
    os.environ["DHUS_PASSWORD"]
except:
    logger.error("The environment variable 'DHUS_PASSWORD' is not set! Exiting...")
    raise KeyError("The environment variable 'DHUS_PASSWORD' is not set!")


In [2]:
# establish connection to Copernicus Scihub
api = SentinelAPI(os.environ["DHUS_USER"], os.environ["DHUS_PASSWORD"])

# search over Fram Strait for S1 imagery (from 1st Dec 2020 to 1st June 2021)
# use custom geojson files containing search polygon
area = './rois/Fram_Strait.geojson'
polygon_path = Path(area).expanduser().absolute()

# search from 1st Dec 2020 to 1st June 2021
starttime = "2020-12-01"
endtime = "2021-06-01"
starttime = parse(starttime, settings={"DATE_ORDER": "YMD"})
endtime = parse(endtime, settings={"DATE_ORDER": "YMD"})

# query for specific product
s1_products = api.query(
                geojson_to_wkt(read_geojson(polygon_path)),
                date=(starttime, endtime),
                platformname="Sentinel-1",
                producttype="GRD",
                )

s1_list_with_overlap = []
s1_s2_overlap = {}

# iterate over query results to check for overlapping S2
for product in s1_products:

    # retrieve some properties of the S1 product
    s1_footprint = s1_products[product]['footprint']
    s1_identifier = s1_products[product]['identifier']
    s1_timestamp = s1_products[product]['beginposition']

    # look for S2 imagery taken 2 hours before or after S1 acquisition
    timedelta = datetime.timedelta(hours = 2) 
    s2_starttime = s1_timestamp - timedelta
    s2_endtime = s1_timestamp + timedelta

    #starttime = parse(starttime, settings={"DATE_ORDER": "YMD"})
    #endtime = parse(endtime, settings={"DATE_ORDER": "YMD"})

    s2_products = api.query(area = s1_footprint,
                            date = (s2_starttime, s2_endtime),
                            platformname="Sentinel-2",
                            processinglevel="Level-1C",
                            cloudcoverpercentage=(0, 30)
                            )
    # if overlapping S2 scenes found, add the S1 id and corresponding S2 query results in dictionary
    if not len(s2_products) == 0:
        s1_list_with_overlap.append(product)
        s1_s2_overlap[product] = s2_products
        
# this list is contains the S1 scenes that have overlapping S2 imagery
print('Found', len(s1_list_with_overlap), 'S1 scenes with overlapping S2 imagery.')
print(s1_list_with_overlap)

# make copy of the original query results
s1_products_with_overlap = s1_products.copy()

# loop over all original query results and only keep the ones that have overlapping S2
for product in s1_products:
    if not product in s1_list_with_overlap:
        del s1_products_overlap[product]

UnauthorizedError: Invalid user name or password. Note that account creation and password changes may take up to a week to propagate to the 'https://apihub.copernicus.eu/apihub/' API URL you are using. Consider switching to 'https://scihub.copernicus.eu/dhus/' instead in the mean time.

In [ ]:
#### Download quicklooks of S1 images that have overlapping S2, then manually select which S1 images to download (through visual inspection)

In [3]:
# download all quicklooks of S1 images that have overlapping S2
api.download_all_quicklooks(s1_products_overlap)

# 'selection' is a list that contains the unique product identifiers of the S1 images that we want to download
# this list is handmade, after visual inspection oi the quicklooks
selection = ['1F44', 'BB21']
s1_uuids_for_download = []

for product in s1_products_overlap:
    for unique_id in selection:
        if unique_id in s1_products_overlap[product]['identifier']:
            s1_uuids_for_download.append(product)    
                
## download the manually selected S1 scenes
## download does not work in Jupyter Notebook, do it locally
# for uuid in s1_uuids_for_download:
    # api.download(uuid)

IndentationError: unexpected indent (2030374187.py, line 20)

In [ ]:
# to download corresponding S2 products, new query is necessary (unless you want to download all S1 and S2 scenes from the first search)
for uuid in s1_uuids_for_download:
    api.query(uuid)
    


